# Chaines de Markhov

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import numpy as np
from scipy import stats
import pysal


table = pd.read_csv('Databases/swaphist.csv', delimiter = ";")
table['Date'] = pd.to_datetime(table['Date'],dayfirst=True)
#table = scipy.delete(table,table.shape[0]-1,0)
print(min(table['1to5']))
np.array(table['1to5'].quantile([k/5 for k in range(0, 5)]))
output_3 = pd.read_csv('Databases/normal_implied_vol_300714.csv', delimiter = ";")
output_2 = pd.read_csv('Databases/normal_implied_vol_010807.csv', delimiter = ";")
#print(output_2)

11.2


# Définition des fonctions

### Calcul de la matrice de transition

In [2]:
#Calcule la place de l'élément dans un des n groupes
'''def place(elem,table,freq):
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/freq
    if table[elem]==xmax : freq-1
    else : return int((table[elem]-xmin)/delta)
    #return min(int((table[elem]-xmin)/delta),freq-1)'''

def place(elem, table, freq, quantiles):
    k = 0
    while k<freq and table[elem]>=quantiles[k]:
        k += 1
    return k-1

def placement(table, freq):
    quantiles = np.array(table.quantile([k/freq for k in range(0, freq)]))
    liste = []
    for elem in table :
        k=1
        while k<freq and elem>=quantiles[k]:
            k += 1
        liste.append(k-1)
    return np.array(liste)

#Calcule la matrice de passage (fréquence de passage d'un groupe à l'autre)
def matrice(table, freq):
    trans = np.zeros((freq,freq))
    n = table.size
    places = placement(table, freq)
    for i in range(0,n-1):
        trans[places[i],places[i+1]] += 1
    for i in range(0,freq):
        sum_ligne = np.sum(trans[i])
        trans[i] = trans[i]/sum_ligne
    return trans



#La matrice de transition est-elle bien stochastique ?
'''test = matrice(table['1to5'],20)
print(test)
for i in range(0,test.shape[0]):
    #print(sum(test[i]))
    if(sum(test[i])!=1.0): print(sum(test[i]))
'''

colors = ['RED','GREEN','BLUE','BLACK','ORANGE','PINK','BROWN','YELLOW','CYAN']
#fig, ax = plt.subplots()
#ax.plot_date(table['Date'], table['1to5'],'-',color='BLACK')
#ax.plot_date(table['Date'],20,'-',color='RED')
#fig.autofmt_xdate()

Les matrices de transitions ne sont pas stochastiques car certains intervalles de valeurs ne sont jamais visités : typiquement, les valeurs élevés atteintes durant le pic de 2008-2013. Les variations sont très importantes lors de cette période et les volatilités font des sauts importants d'une semaine à l'autre.

Il faudra prendre cela en compte lorsque l'on conduira les tests. En effet, les tests doivent être effectués sur des matrices stochastiques. On supprimera donc les colonnes et lignes vides.

### Fonctions auxiliaires pour le calcul des statistiques de test

In [3]:
#Une fonction pour supprimer les colonnes et lignes qui ne contiennent que des zeros
def zero(mat):
    liste = []
    for i in range(0,mat.shape[0]-1):
        if(sum(mat[i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,0)
    liste = []
    for i in range(0,mat.shape[1]-1): 
        if(sum(mat[:,i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,1)
    return mat

In [4]:
def A(table,i,freq,T):
    a=0
    delta = table.size/T
    tmp=0
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        #tmp =+ n(sous_table,i,freq)
        if n(sous_table,i,freq)!=0 : a+=1
    return a

def B(MT,i):
    B = []
    for j in range(0,MT.shape[1]):
        if MT[i,j]!=0 : B.append(j)
    return B

#elements de table appartenant à la i-ème classe
def n(sous_table,i,freq):
    tot = 0
    quantiles = np.array(sous_table.quantile([k/freq for k in range(0, freq)]))
    for elem in range(0,sous_table.size) :
        if place(elem,sous_table,freq,quantiles)==i : tot += 1
    return tot

def d(table,i,T,freq):
    delta = table.size/freq
    di = []
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        if n(sous_table,i,T)>0 : di.append(t)
    return di

def stat(table, MT, sous_table, sous_MT, T):
    dg_lbt = 0
    tmp = 0
    for i in range(0,MT.shape[0]):
        #print("i : ",i)
        ni = n(sous_table,i, T)
        if ni>0 :
            Bi = B(MT,i)
            for j in Bi:
                tmp += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
                #if MT[i,j]!=0 : stat += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
    return tmp

In [5]:
#Les matrices de transition de la table est-elle homogène ?
def homog(table, freq=50, T=5):
    for i,elem in enumerate(table):
        if i>0:
            Q,p = homog_temp(table[elem].values,freq,T)
            m1 = homog_int(table[elem].values,freq,T)
            m2 = homog_reg(table[elem].values,freq,T)
            print("p-valeur : ",p, " - résultat du test par intervalles : ",m1," - résultat du test par régions : ",m2)
            rep = p>0.05 and m1>0.5 and m2>0.5
            if rep : print("La série de valeur pour le swaption ",elem," est homogène")
            else : print("La série de valeur pour le swaption ",elem," n'est pas homogène")
    return 0

## Les matrices de transition sont elles bien homogènes ?

Pour que la matrice de transition calculée à partir des fréquences de transition ait un sens, il fait s'assurer qu'elle est bien homogène en temps. On va donc mener plusieurs test pour vérifier cette hypothèse lourde.

### Test d'homogénéité des intervalles par rapport à la matrice de transition globale

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On vérifie que, si on calcule la matrice de transition sur chacun des intervalles, on retrouve bien la matrice de transition calculée sur toutes les valeurs de la série par un test du khi-2.


Notre hypothèse nulle est donc : $H_0 = \{\forall t, \le T, \forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [6]:
def homog_temp(table, freq, T=5):
    N = table.size
    Q = 0
    delta = N/T
    MT = matrice(table, freq)
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        sous_MT = matrice(sous_table, freq)
        Q += stat(table,MT,sous_table,sous_MT,T)
    
    dg_lib = 0
    for i in range(0, MT.shape[0]):
        ai = A(table,i,freq,T)
        if ai>0 : dg_lib += (len(B(MT,i))-1)*(ai-1)
    print("Degrés de liberté : ",dg_lib)
    return Q, 1-stats.chi2.cdf(Q,dg_lib)

### Test d'homogénéité par intervalle

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On va effectuer $T$ tests pour vérifier si chacune des $T$ matrices de transitions sont bien les mêmes que la matrice de transition globale.


Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t <T$ : $H_0^t = \{\forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [7]:
def homog_int(table, freq, T=5):
    N = table.size
    Q = 0
    delta = N/T
    moy = 0
    MT = matrice(table, freq)
    for i in range(0,freq):
        Q = 0
        Di = d(table,i,T,freq)
        Bi = B(MT,i)
        for t in Di:
            ni = n(table,i,T)
            sous_table = table[int(t*delta): int((t+1)*delta)]
            sous_table.index = range(sous_table.shape[0])
            sous_MT = matrice(sous_table,freq)
            for j in Bi:
                Q += ni*(MT[i,j]-sous_MT[i,j])**2/MT[i,j]
        dg_lib = (len(Di)-1)*(len(Bi)-1)
        p = 1-stats.chi2.cdf(Q,dg_lib)

        print("Q : ",Q, " - dg de lbte : ",dg_lib)
        print("p-valeur du test sur le ",t,"-ième intervalle : ",p)
        if p>0.05 : moy += 1
    
    print(moy," intervalles de temps sur ",T," sont homogènes")    
    return moy/T


    ''' for i in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        sous_MT = matrice(sous_table, freq)
        Q = stat(table,MT,sous_table,sous_MT,freq)
        dg_lib = (len(B(MT,i))-1)*n(table,i,freq)
        #for i in range(0, MT.shape[0]-1):
            #print(B(MT,i),A(table,i,freq,T))
            #dg_lib += (len(B(MT,i))-1)*(A(table,i,freq,T)-1)
        p = 1-stats.chi2.cdf(Q,dg_lib)'''

### Test région contre région

On subdivise toujours notre série de valeurs en un nombre $T$ d'intervalles. Pour finir, on vérifie si les matrices de transitions de deux périodes sont identiques.

Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t,u <T$ : $H_0^{t,u} = \{\forall i,j, p_{i,j}^t (t) = p_{i,j}^u\}$

In [8]:
def homog_reg(table, freq, T=5):
    N = table.size
    Q = 0
    delta = N/T
    moy = 0
    t,u = 0,0
    for t in range(0,T):
        for u in range(t+1,T):
            sous_table1 = table[int(t*delta): int((t+1)*delta)]
            sous_table2 = table[int(u*delta): int((u+1)*delta)]
            sous_table1.index = range(int((t+1)*delta)-int(t*delta))
            sous_table2.index = range(int((u+1)*delta)-int(u*delta))
            sous_MT1 = matrice(sous_table1, freq)
            sous_MT2 = matrice(sous_table2, freq)
            Q = stat(sous_table1,sous_MT1,sous_table2,sous_MT2,T)
            dg_lib = (len(Di)-1)*(len(Bi)-1)
            MT = matrice(sous_table1, freq)
            #for i in range(0, MT.shape[0]-1):
                #dg_lib += max((len(B(MT,i))-1)*(A(table,i,freq,T)-1),0)
            p = 1-stats.chi2.cdf(Q,dg_lib)
            print("Q : ",Q," - dg lib : ",dg_lib)
            print("p-valeur du test entre le ",t+1,"-ième et le ",u+1,"-ième intervalle : ",p)
            if p>0.05 : moy += 1
    
    print(moy," paires d'intervalles de temps sur ",T," intervalles sont homogènes")    
    return 2*moy/(T*(T-1))

SyntaxError: invalid syntax (<ipython-input-8-19b0758eb3bf>, line 16)

## Deux matrices de transition sont-elles égales ?

Maintenant, on peut tester si deux matrices de transition sont égales. On appliquera la statistique de test précédement définie aux deux matrices.

Notre hypothèse nulle est donc, si on note $(p_{i,j}^0)$ et $(p_{i,j}^1)$ les coefficents de chacune des deux matrices de transition : $H_0 = \{\forall i,j, p_{i,j}^0 = p_{i,j}^1\}$

# Tests sur toute la période 2001-2016

## Homogénéité

In [10]:
homog_temp(table['10to5'],25)

Degrés de liberté :  420


(992.65539335059418, 0.0)

In [13]:
homog_int(table['1to5'],20,T=5)

Q :  96.0068000638  - dg de lbte :  4
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  1349.88695652  - dg de lbte :  16
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  502.620039683  - dg de lbte :  12
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  243.60831608  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  438.424789359  - dg de lbte :  16
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0 

0.0

In [12]:
homog_reg(table['1to5'],20)

NameError: name 'homog_reg' is not defined

Comme on pouvait s'y attendre, les matrices de transition sur la période 2001-2016 ne sont homogène dans le temps pour aucun des swaptions. Nous allons donc restreindre notre études à de plus petits intervalles.

# Test sur la période 2001-2008

In [14]:
tab1 = table[0:292]

In [15]:
homog_reg(tab1['1to5'],10)

NameError: name 'homog_reg' is not defined

In [16]:
homog_int(tab1['1to5'],20)

Q :  752.388888889  - dg de lbte :  12
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  nan  - dg de lbte :  12
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  366.85  - dg de lbte :  20
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  nan  - dg de lbte :  12
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  280.625  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -5
p-valeur du

0.0

In [17]:
homog_temp(tab1['1to5'],10,T=5)

Degrés de liberté :  112


(274.7375787319217, 1.1102230246251565e-15)

## Test sur la période 2008-2014

In [18]:
tab2 = table[292:658]
tab2.index = range(tab2.shape[0])

In [19]:
homog_reg(tab2['1to5'],20)

NameError: name 'homog_reg' is not defined

In [20]:
homog_int(tab2['1to5'],20)

Q :  204.4  - dg de lbte :  4
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  542.068452381  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  457.714506173  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  219.957111111  - dg de lbte :  16
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  907.527777778  - dg de lbte :  16
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -5
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -5
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de

0.0

In [21]:
homog_temp(tab2['1to5'],10,T=5)

Degrés de liberté :  96


(281.24660076254941, 0.0)

## Test sur la période 2014-2016

In [22]:
tab3 = table[658:]
tab3.index = range(tab3.shape[0])

In [23]:
homog_int(tab3['1to5'],20)

Q :  161.5  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  144.0  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  nan  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  nan  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -i

0.0

# Output

In [29]:
#print(table)
table['Date'] = pd.to_datetime(table['Date'],dayfirst=True)
dates = table['Date'][table['Date'].dt.year==2011]

#resultat= np.zeros((51,9))
swaptions = output_2.columns[2:]

In [30]:
#Calcule la moyenne des matrices de transition sur les 1000 itérations

def mat_output(output,elem,freq):
    matrice_freq = np.zeros((freq,freq))
    for i in range(1,1000):
        sous_table = output[output.Trial.values==i][elem]
        sous_table.index = range(sous_table.shape[0])
        matrice_freq += matrice(sous_table,freq)

    return matrice_freq/1000

In [31]:
def egales(table_in, table_out, elem, freq=10):
    MT1 = matrice(table_in[elem], freq)
    MT2 = mat_output(table_out, elem, freq)
    table_mean = table_out.groupby(table_out.Trial).mean()
    table_mean.index = range(table_mean.shape[0])
    Q = stat(table_in[elem],MT1,table_mean[elem],MT2,freq)
    dg_lib = 0
    for i in range(0, MT1.shape[0]-1):
        dg_lib += (len(B(MT1,i))-1)
    return Q, 1-stats.chi2.cdf(Q,dg_lib)


In [33]:
#Sur toute la durée

for elem in table.columns[1:]:
    print(elem,egales(table,output_2,elem))

1to5 (966.50229477241987, 0.0)
1to10 (931.48942203418073, 0.0)
1to20 (659.41870619241456, 0.0)
5to5 (745.76113124878646, 0.0)
5to10 (691.54168363510655, 0.0)
5to20 (747.21741064506682, 0.0)
10to5 (867.89978087652298, 0.0)
10to10 (988.8019963303924, 0.0)
10to20 (796.91305143779289, 0.0)
